In [1]:
import pandas as pd
import numpy as np
import os
import datetime
os.getcwd()

# Note email cpm updated based on the Q4 client number
# Flipp hosted not included in wide version

'/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2019Q2'

In [2]:
TMR_up_to_2019Q1=pd.read_csv("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2019Q1/output/BL_MMM_long_cumu_up_to_2019Q1_JL_2019-09-10.csv",dtype=str)
TMR_up_to_2019Q1['impression']=TMR_up_to_2019Q1['impression'].astype(float).astype(int)
TMR_up_to_2019Q1['click']=TMR_up_to_2019Q1['click'].astype(float).astype(int)
TMR_up_to_2019Q1['cost']=TMR_up_to_2019Q1['cost'].astype(float)


In [3]:
TMR_up_to_2019Q1_no_DMA=TMR_up_to_2019Q1.groupby(["week date","media","submedia","placement"])['impression','click','cost'].sum().reset_index()
print(TMR_up_to_2019Q1_no_DMA['week date'].min())
print(TMR_up_to_2019Q1_no_DMA['week date'].max())
# TMR_up_to_2019Q1_no_DMA

2016-09-25
2019-04-28


In [4]:
TMR_up_to_2019Q1_no_DMA.shape

(6189, 7)

# TV

In [5]:
TV_logs=pd.ExcelFile("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2019Q2/input_2019Q2/Q2_National_TV_Print Circ_TMR Reports 10.10.19.xlsx")
print(TV_logs.sheet_names)
TV_logs=TV_logs.parse("Q2 Nat Cable 2019",dtype=str,skiprows=1)
TV_logs.head(2)

['Q2 Nat Cable 2019', 'Q2 Print Circ 2019']


,Campaign,Status,Data Source,Market,Network,Media Type,Demographic,Week Of,Air Date,Air Day,Air Time,Spot Length,Aired Prg Name,Rating,IMPs (Units),Net Cost,ISCI Length,Air ISCI,Cmml Title
0,Q2 2019,Post,Nielsen Keeping Trac,National,BET,Cable,W25-54,2019-05-27 00:00:00,2019-05-28 00:00:00,Tuesday,23:27:57,30,GAMES PEOPLE PLAY,0.23,138207,0,30,LGTS1115000,19May/BL/L&G/MemDay/Generic
1,Q2 2022,Post,Nielsen Keeping Trac,National,BET,Cable,W25-54,2019-06-17 00:00:00,2019-06-23 00:00:00,Sunday,17:59:00,30,2019 BETA RED CARPET HH,0.24,148366,0,30,LGTS1126000,23June/BL/L&G/JulyFourthPatioCle


In [6]:
TV_logs=TV_logs[["Air Date","Net Cost",'IMPs (Units)','Demographic','Network']]
print(TV_logs['Demographic'].unique())
TV_logs=TV_logs[TV_logs['Demographic']=="W25-54"]
TV_logs=TV_logs[TV_logs['Air Date']!="nan"]
print(TV_logs['Demographic'].unique())

TV_logs['Net Cost']=TV_logs['Net Cost'].astype(float)
TV_logs['IMPs (Units)']=TV_logs['IMPs (Units)'].astype(float)
TV_logs['IMPs (Units)']=TV_logs['IMPs (Units)'].fillna(0).astype(int)
TV_logs=TV_logs.rename(columns={"Net Cost":"cost","IMPs (Units)":"impression"})

TV_logs['date']=TV_logs['Air Date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").date())
TV_logs=TV_logs[(TV_logs['date']<=datetime.date(2019,8,3)) & (TV_logs['date']>=datetime.date(2019,5,5))]


['W25-54']
['W25-54']


In [7]:
TV_logs['cost'].sum()

1697094.7000000002

In [8]:
TV_logs['impression'].sum()

111259722

In [9]:
def get_week_end_date_BL(x):
    if x.weekday()==6:
        y=x+datetime.timedelta(days=6)
    else:
        y=x+datetime.timedelta(days=5-x.weekday())
    return y

In [10]:
TV_logs['week_end_dt']=TV_logs['date'].apply(lambda x: get_week_end_date_BL(x))
TV_logs['week date']=TV_logs['week_end_dt'].apply(lambda x: x-datetime.timedelta(days=6))
TV_TMR_data=TV_logs.groupby(['week date','Network'])[['cost','impression']].sum().reset_index()
TV_TMR_data=TV_TMR_data.rename(columns={"Network":"placement"})
TV_TMR_data['media']="TV"
TV_TMR_data['submedia']="National"
TV_TMR_data['click']=0
TV_TMR_data=TV_TMR_data[TMR_up_to_2019Q1_no_DMA.columns.tolist()]
TV_TMR_data['week date']=TV_TMR_data['week date'].astype(str)

In [11]:
print(TV_TMR_data['week date'].min())
print(TV_TMR_data['week date'].max())

2019-05-19
2019-06-30


In [12]:
TV_TMR_data.head(2)

,week date,media,submedia,placement,impression,click,cost
0,2019-05-19,TV,National,BRAV,226629,0,0.00
1,2019-05-19,TV,National,E!,1706420,0,32900.95


# Others

In [13]:
Others_TMR_week=pd.read_excel("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2019Q2/input_2019Q2/DatoramaData_2019Q2_1113.xlsx",
                              dtype=str,sheet_name="BL_2019Q2_TMR")
Others_TMR_week=Others_TMR_week.rename(columns={"Week":"week date","Media":"media","Submedia":"submedia","Placement":"placement",
                                               "Impressions":"impression","Clicks":"click","Media Cost":"cost"})
Others_TMR_week['week date']=Others_TMR_week['week date'].apply(lambda x: x[:10])
Others_TMR_week['impression']=Others_TMR_week['impression'].fillna(0).astype(float).astype(int)
Others_TMR_week['click']=Others_TMR_week['click'].fillna(0).astype(float).astype(int)
Others_TMR_week['cost']=Others_TMR_week['cost'].fillna(0).astype(float)


### Revise Emails

In [14]:
Others_TMR_week_1=Others_TMR_week[Others_TMR_week['media']!="Email"]
Others_TMR_week_2=Others_TMR_week[Others_TMR_week['media']=="Email"]
Others_TMR_week_2['impression'].sum(),Others_TMR_week_2['cost'].sum()

(2033659691, 926910.6031155817)

In [15]:
# Based on the 2019Q1 number
# Email from Kipp, Bryan 2019 July 19, Friday 5:17 p.m.
email_cost_client=494229
email_cost_impression=2075000494
new_email_c_p_impr=email_cost_client/email_cost_impression
print(new_email_c_p_impr)
Others_TMR_week_2['cost']=Others_TMR_week_2['impression']*new_email_c_p_impr
print(new_email_c_p_impr*1000)

0.00023818259389773427
0.23818259389773427


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [16]:
print(Others_TMR_week_2['week date'].min())
print(Others_TMR_week_2['week date'].max())

Others_TMR_week=Others_TMR_week_1.append(Others_TMR_week_2)

2019-05-05
2019-07-28


## Add Print Circ

In [17]:
updated_circ_2019Q2_Kayla=pd.read_excel("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2019Q2/input_2019Q2/Q2_National_TV_Print Circ_TMR Reports 10.10.19.xlsx",
                                        dtype=str,skiprows=1,sheet_name="Q2 Print Circ 2019")

In [18]:
updated_circ_2019Q2_Kayla=updated_circ_2019Q2_Kayla[['week_start_Sunday','Circulation actual delivers','Circulation total cost']]
updated_circ_2019Q2_Kayla=updated_circ_2019Q2_Kayla.rename(columns={"week_start_Sunday":"week date",
                                                                   "Circulation actual delivers":"impression",
                                                                   "Circulation total cost":"cost"})
updated_circ_2019Q2_Kayla['impression']=updated_circ_2019Q2_Kayla['impression'].astype(int)
updated_circ_2019Q2_Kayla['cost']=updated_circ_2019Q2_Kayla['cost'].astype(float)

updated_circ_2019Q2_Kayla['week date']=updated_circ_2019Q2_Kayla['week date'].apply(lambda x: x.split(" ")[0])
updated_circ_2019Q2_Kayla['click']=0
updated_circ_2019Q2_Kayla['media']="Circulation"
updated_circ_2019Q2_Kayla['submedia']="xx"
updated_circ_2019Q2_Kayla['placement']="xx"
updated_circ_2019Q2_Kayla=updated_circ_2019Q2_Kayla[Others_TMR_week.columns.tolist()]



In [19]:
updated_circ_2019Q2_Kayla.head(2)

,week date,media,submedia,placement,impression,click,cost
0,2019-05-05,Circulation,xx,xx,0,0,0.0
1,2019-05-12,Circulation,xx,xx,0,0,0.0


In [20]:
for col in Others_TMR_week.columns.tolist():
    print(col,Others_TMR_week[col].apply(type).unique())
    
Others_TMR_week=Others_TMR_week[Others_TMR_week['media']!="Circulation"] 
Others_TMR_week=Others_TMR_week.append(updated_circ_2019Q2_Kayla)

week date [<class 'str'>]
media [<class 'str'>]
submedia [<class 'str'>]
placement [<class 'str'>]
impression [<class 'int'>]
click [<class 'int'>]
cost [<class 'float'>]


# adjust media numbers in house

In [21]:
# Circulation

df_circ_in_quarter=Others_TMR_week[Others_TMR_week['media']=="Circulation"]
Others_TMR_week=Others_TMR_week[Others_TMR_week['media']!="Circulation"]

old_impr_total_circ=df_circ_in_quarter['impression'].sum()
old_click_total_circ=df_circ_in_quarter['click'].sum()
old_cost_total_circ=df_circ_in_quarter['cost'].sum()

print("df_circ_in_quarter['impression'].sum():", old_impr_total_circ)
print("df_circ_in_quarter['click'].sum():", old_click_total_circ)
print("df_circ_in_quarter['cost'].sum():", old_cost_total_circ)

new_clients_impr=82346555
new_clients_click=0
new_clients_cost=4706007

df_circ_in_quarter['impression']=df_circ_in_quarter['impression'].apply(lambda x: int(x/old_impr_total_circ*new_clients_impr))
# df_circ_in_quarter['click']=df_circ_in_quarter['click'].apply(lambda x: int(x/old_click_total_circ*new_clients_click))
df_circ_in_quarter['cost']=df_circ_in_quarter['cost'].apply(lambda x: x/old_cost_total_circ*new_clients_cost)

print("df_circ_in_quarter['impression'].sum():", df_circ_in_quarter['impression'].sum(),"new")
# print("df_circ_in_quarter['click'].sum():", df_circ_in_quarter['click'].sum(),"new")
print("df_circ_in_quarter['cost'].sum():", df_circ_in_quarter['cost'].sum(),"new")


Others_TMR_week=Others_TMR_week.append(df_circ_in_quarter)

df_circ_in_quarter['impression'].sum(): 82281119
df_circ_in_quarter['click'].sum(): 0
df_circ_in_quarter['cost'].sum(): 4917129.82
df_circ_in_quarter['impression'].sum(): 82346551 new
df_circ_in_quarter['cost'].sum(): 4706007.0 new


In [22]:
# Flipp

df_flipp_in_quarter_hosted=Others_TMR_week[(Others_TMR_week['submedia']=="Flipp") & (Others_TMR_week['placement']=="Hosted")]
df_flipp_in_quarter_nonhosted=Others_TMR_week[(Others_TMR_week['submedia']=="Flipp") & (Others_TMR_week['placement']!="Hosted")]
Others_TMR_week=Others_TMR_week[(Others_TMR_week['submedia']!="Flipp")]


old_impr_hosted_flipp=df_flipp_in_quarter_hosted['impression'].sum()
old_click_hosted_flipp=df_flipp_in_quarter_hosted['click'].sum()
old_cost_hosted_flipp=df_flipp_in_quarter_hosted['cost'].sum()

old_impr_Nonhosted_flipp=df_flipp_in_quarter_nonhosted['impression'].sum()
old_click_Nonhosted_flipp=df_flipp_in_quarter_nonhosted['click'].sum()
old_cost_Nonhosted_flipp=df_flipp_in_quarter_nonhosted['cost'].sum()

print("df_flipp_in_quarter_hosted['impression'].sum():", old_impr_hosted_flipp)
print("df_flipp_in_quarter_hosted['click'].sum():", old_click_hosted_flipp)
print("df_flipp_in_quarter_hosted['cost'].sum():", old_cost_hosted_flipp)

print("df_flipp_in_quarter_nonhosted['impression'].sum():", old_impr_Nonhosted_flipp)
print("df_flipp_in_quarter_nonhosted['click'].sum():", old_click_Nonhosted_flipp)
print("df_flipp_in_quarter_nonhosted['cost'].sum():", old_cost_Nonhosted_flipp)

new_clients_impr_hosted_flipp=3281334
new_clients_click_hosted_flipp=2740351
# new_clients_cost_hosted_flipp=0
df_flipp_in_quarter_hosted['impression']=df_flipp_in_quarter_hosted['impression'].apply(lambda x: int(x/old_impr_hosted_flipp*new_clients_impr_hosted_flipp))
df_flipp_in_quarter_hosted['click']=df_flipp_in_quarter_hosted['click'].apply(lambda x: int(x/old_click_hosted_flipp*new_clients_click_hosted_flipp))
# df_flipp_in_quarter_hosted['cost']=df_flipp_in_quarter_hosted['cost'].apply(lambda x: x/old_cost_total_circ*new_clients_cost)
print("df_flipp_in_quarter_hosted['impression'].sum():", df_flipp_in_quarter_hosted['impression'].sum(),"new")
print("df_flipp_in_quarter_hosted['click'].sum():", df_flipp_in_quarter_hosted['click'].sum(),"new")
print("df_flipp_in_quarter_hosted['cost'].sum():", df_flipp_in_quarter_hosted['cost'].sum(),"new")

############
new_clients_impr_Nonhosted_flipp=2415572
new_clients_click_Nonhosted_flipp=971281
new_clients_cost_Nonhosted_flipp=356158
df_flipp_in_quarter_nonhosted['impression']=df_flipp_in_quarter_nonhosted['impression'].apply(lambda x: int(x/old_impr_Nonhosted_flipp*new_clients_impr_Nonhosted_flipp))
df_flipp_in_quarter_nonhosted['click']=df_flipp_in_quarter_nonhosted['click'].apply(lambda x: int(x/old_click_Nonhosted_flipp*new_clients_click_Nonhosted_flipp))
df_flipp_in_quarter_nonhosted['cost']=df_flipp_in_quarter_nonhosted['cost'].apply(lambda x: x/old_cost_Nonhosted_flipp*new_clients_cost_Nonhosted_flipp)
print("df_flipp_in_quarter_nonhosted['impression'].sum():", df_flipp_in_quarter_nonhosted['impression'].sum(),"new")
print("df_flipp_in_quarter_nonhosted['click'].sum():", df_flipp_in_quarter_nonhosted['click'].sum(),"new")
print("df_flipp_in_quarter_nonhosted['cost'].sum():", df_flipp_in_quarter_nonhosted['cost'].sum(),"new")


Others_TMR_week=Others_TMR_week.append(df_flipp_in_quarter_hosted).append(df_flipp_in_quarter_nonhosted)


df_flipp_in_quarter_hosted['impression'].sum(): 3277907
df_flipp_in_quarter_hosted['click'].sum(): 2833954
df_flipp_in_quarter_hosted['cost'].sum(): 0.0
df_flipp_in_quarter_nonhosted['impression'].sum(): 2417806
df_flipp_in_quarter_nonhosted['click'].sum(): 1242041
df_flipp_in_quarter_nonhosted['cost'].sum(): 360191.8899999996
df_flipp_in_quarter_hosted['impression'].sum(): 3281327 new
df_flipp_in_quarter_hosted['click'].sum(): 2740345 new
df_flipp_in_quarter_hosted['cost'].sum(): 0.0 new
df_flipp_in_quarter_nonhosted['impression'].sum(): 2415561 new
df_flipp_in_quarter_nonhosted['click'].sum(): 971272 new
df_flipp_in_quarter_nonhosted['cost'].sum(): 356158.0 new


In [23]:
Others_TMR_week['media'].unique()

array(['Digital', 'Email', 'Circulation'], dtype=object)

In [24]:
# Email confirmed from Dom on 2019 11 15, 1:44 pm
df_in_quarter_email=Others_TMR_week[(Others_TMR_week['media']=="Email")]
Others_TMR_week=Others_TMR_week[(Others_TMR_week['media']!="Email")]

old_impr_email=df_in_quarter_email['impression'].sum()
old_click_email=df_in_quarter_email['click'].sum()
old_cost_email=df_in_quarter_email['cost'].sum()


print("df_in_quarter_email['impression'].sum():", old_impr_email)
print("df_in_quarter_email['click'].sum():", old_click_email)
print("df_in_quarter_email['cost'].sum():", old_cost_email)

# new_clients_impr_email=2.04B
# new_clients_click_email=/
new_clients_cost_email=530690

# df_in_quarter_email['impression']=df_in_quarter_email['impression'].apply(lambda x: int(x/*))
# df_in_quarter_email['click']=df_in_quarter_email['click'].apply(lambda x: int(x/*))
df_in_quarter_email['cost']=df_in_quarter_email['cost'].apply(lambda x: x/old_cost_email*new_clients_cost_email)

print("df_in_quarter_email['impression'].sum():", df_in_quarter_email['impression'].sum(),"new")
print("df_in_quarter_email['click'].sum():", df_in_quarter_email['click'].sum(),"new")
print("df_in_quarter_email['cost'].sum():", df_in_quarter_email['cost'].sum(),"new")

Others_TMR_week=Others_TMR_week.append(df_in_quarter_email)

df_in_quarter_email['impression'].sum(): 2033659691
df_in_quarter_email['click'].sum(): 16128461
df_in_quarter_email['cost'].sum(): 484382.34030764474
df_in_quarter_email['impression'].sum(): 2033659691 new
df_in_quarter_email['click'].sum(): 16128461 new
df_in_quarter_email['cost'].sum(): 530690.0 new


# Cumulative Q2 TMR

In [25]:
Q2_2019_only_TMR=Others_TMR_week.append(TV_TMR_data)

Cum_Q2_2019_TMR=TMR_up_to_2019Q1.append(Q2_2019_only_TMR)

/home/jian/.local/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [26]:
try:
    os.stat("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2019Q2/output/")
except:
    os.mkdir("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2019Q2/output/")

Q2_2019_only_TMR.to_csv("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2019Q2/output/BL_MMM_2019Q2_Only_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
Cum_Q2_2019_TMR.to_csv("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2019Q2/output/BL_MMM_long_cumu_up_to_2019Q2_JL_"+str(datetime.datetime.now().date())+".csv",index=False)



In [27]:
Cum_Q2_2019_TMR[Cum_Q2_2019_TMR['media']=="Circulation"]['submedia'].unique()

array(['FSI+Instore', 'Valassis', 'xx'], dtype=object)

In [28]:
Cum_Q2_2019_TMR.shape

(3484039, 8)

In [29]:
set([datetime.datetime.strptime(x,"%Y-%m-%d").weekday() for x in Cum_Q2_2019_TMR['week date'].unique().tolist()])

{6}

In [30]:
df_temp_check=Cum_Q2_2019_TMR[(Cum_Q2_2019_TMR['submedia']=="Flipp") & (Cum_Q2_2019_TMR['placement']!="Hosted")]
df_temp_check['placement'].unique()

array(['Flipp App', 'Native Network'], dtype=object)

In [31]:
df_temp_check=Cum_Q2_2019_TMR[(Cum_Q2_2019_TMR['submedia']=="Flipp") & (Cum_Q2_2019_TMR['placement']=="Hosted")]
df_temp_check['placement'].unique()

array(['Hosted'], dtype=object)

# Wide

In [32]:
# seperate the hosted flipp
Cum_Q2_2019_TMR_FlippHost=Cum_Q2_2019_TMR[(Cum_Q2_2019_TMR['submedia']=="Flipp") & (Cum_Q2_2019_TMR['placement']=="Hosted")]
Cum_Q2_2019_TMR_FlippHost['submedia']="Flipp(hosted)"

Cum_Q2_2019_TMR_FlippNonHost=Cum_Q2_2019_TMR[(Cum_Q2_2019_TMR['submedia']=="Flipp") & (Cum_Q2_2019_TMR['placement']!="Hosted")]
Cum_Q2_2019_TMR_FlippNonHost['submedia']="Flipp(nonhosted)"


Cum_Q2_2019_TMR=Cum_Q2_2019_TMR[(Cum_Q2_2019_TMR['submedia']!="Flipp")]
Cum_Q2_2019_TMR['submedia']=np.where(Cum_Q2_2019_TMR['media']=="Circulation","xx",Cum_Q2_2019_TMR['submedia'])


Cum_Q2_2019_TMR=Cum_Q2_2019_TMR.append(Cum_Q2_2019_TMR_FlippHost).append(Cum_Q2_2019_TMR_FlippNonHost)


########################

data_media_national=Cum_Q2_2019_TMR.groupby(['week date','media'])['impression','click','cost'].sum().reset_index()
data_submedia_national=Cum_Q2_2019_TMR.groupby(['week date','media','submedia'])['impression','click','cost'].sum().reset_index()
data_submedia_national['media_submedia']=data_submedia_national['media']+"-"+data_submedia_national['submedia']
data_submedia_national=data_submedia_national[['week date','media_submedia','impression','click','cost']]


data_media_national_wide=data_media_national[['week date']].drop_duplicates()
for col in data_media_national.columns.tolist()[-3:]:
    df=data_media_national[['week date','media']+[col]].pivot(index='week date',columns='media',values=col).reset_index()
    df_new_col_list=df.columns.tolist()
    df_new_col_list.remove('week date')
    df_new_col_dict={}
    for new_col in df_new_col_list:
        df_new_col_dict.update({new_col:new_col+"_"+col})
    df=df.rename(columns=df_new_col_dict)
    data_media_national_wide=pd.merge(data_media_national_wide,df,on='week date',how="outer")
data_media_national_wide=data_media_national_wide.fillna(0)
data_media_national_wide=data_media_national_wide.drop_duplicates()
#############

data_submedia_national_wide=data_submedia_national[['week date']].drop_duplicates()
for col in data_submedia_national.columns.tolist()[-3:]:
    df=data_submedia_national[['week date','media_submedia']+[col]].pivot(index='week date',columns='media_submedia',values=col).reset_index()
    df_new_col_list=df.columns.tolist()
    df_new_col_list.remove('week date')
    df_new_col_dict={}
    for new_col in df_new_col_list:
        df_new_col_dict.update({new_col:new_col+"_"+col})
    df=df.rename(columns=df_new_col_dict)
    data_submedia_national_wide=pd.merge(data_submedia_national_wide,df,on='week date',how="outer")
data_submedia_national_wide=data_submedia_national_wide.fillna(0)
data_submedia_national_wide=data_submedia_national_wide.drop_duplicates()


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [33]:
Cum_Q2_2019_TMR['click'].apply(lambda x: type(x)).unique()

array([<class 'int'>], dtype=object)

In [34]:
including_weeks=[str(datetime.date(2019,8,3)-datetime.timedelta(days=7*x+6)) for x in range(104)]


data_media_national_wide=data_media_national_wide.sort_values("week date")
data_media_national_wide=data_media_national_wide[data_media_national_wide['week date'].isin(including_weeks)]

data_submedia_national_wide=data_submedia_national_wide.sort_values("week date")
data_submedia_national_wide=data_submedia_national_wide[data_submedia_national_wide['week date'].isin(including_weeks)]



In [35]:
Cum_Q2_2019_TMR.head(2)

,cleaned dma,click,cost,impression,media,placement,submedia,week date
0,"Los Angeles, CA",6687,309.818159,679747,Email,xx,xx,2016-09-25
1,"Dallas-Ft. Worth, TX",3185,147.561605,323753,Email,xx,xx,2016-09-25


## Sales

In [36]:
sales_data=pd.read_csv("/home/jian/BiglotsCode/outputs/combined_sales_long_2019-10-26.csv",dtype=str)
sales_data=sales_data[sales_data['week_end_date']<="2019-08-03"]
sales_data['week_end_date']=sales_data['week_end_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
sales_data['sales']=sales_data['sales'].astype(float)
sales_data['transactions']=sales_data['transactions'].astype(float)
sales_data['week date']=sales_data['week_end_date'].apply(lambda x: x-datetime.timedelta(days=6))
sales_data_naitonal=sales_data.groupby(['week date'])['sales','transactions'].sum().reset_index()
sales_data_naitonal=sales_data_naitonal.set_index(['week date'])
sales_data_naitonal.at[datetime.date(2017,4,30),'transactions']=(sales_data_naitonal.at[datetime.date(2017,4,23),'transactions']+sales_data_naitonal.at[datetime.date(2017,5,7),'transactions'])/2
sales_data_naitonal=sales_data_naitonal.reset_index()
sales_data_store_counts=sales_data[sales_data['sales']>0]
sales_data_store_counts=sales_data_store_counts.groupby(['week date'])['location_id'].count().to_frame().reset_index().rename(columns={"location_id":"store_counts"})

sales_data_naitonal=pd.merge(sales_data_naitonal,sales_data_store_counts,on="week date",how="outer")
sales_data_naitonal['week date']=sales_data_naitonal['week date'].astype(str)

In [37]:
data_media_national_wide=pd.merge(data_media_national_wide,sales_data_naitonal,on="week date",how="left")
data_submedia_national_wide=pd.merge(data_submedia_national_wide,sales_data_naitonal,on="week date",how="left")

## Promotion

In [38]:
Rewards_Promotion_list=[datetime.date(2016,10,1),datetime.date(2016,10,2),
                        datetime.date(2017,1,21),datetime.date(2017,1,22),
                        datetime.date(2017,4,1),datetime.date(2017,4,2),
                        datetime.date(2017,7,8),datetime.date(2017,7,9),
                        datetime.date(2017,9,30),datetime.date(2017,10,1),
                        datetime.date(2018,1,20),datetime.date(2018,1,21),
                        datetime.date(2018,4,7),datetime.date(2018,4,8),
                        datetime.date(2018,7,7),datetime.date(2018,7,8),
                        datetime.date(2018,9,29),datetime.date(2018,9,30),
                        datetime.date(2019,1,26),datetime.date(2019,1,27),  # Changed here
                        datetime.date(2019,4,6),datetime.date(2019,4,7),
                        datetime.date(2019,7,13),datetime.date(2019,7,14)]

df_Rewards_Promotion=pd.DataFrame({"Date":Rewards_Promotion_list},index=range(len(Rewards_Promotion_list)))
df_Rewards_Promotion['weekday']=df_Rewards_Promotion['Date'].apply(lambda x: x.weekday())
df_Rewards_Promotion['week date']=np.where(df_Rewards_Promotion['weekday']==6,df_Rewards_Promotion['Date'],df_Rewards_Promotion['Date']-datetime.timedelta(days=6))

del df_Rewards_Promotion['Date']

df_Rewards_Promotion_Sunday=df_Rewards_Promotion[df_Rewards_Promotion['weekday']==6]
df_Rewards_Promotion_Sunday['Sunday_rewards_ind']=1
del df_Rewards_Promotion_Sunday['weekday']
df_Rewards_Promotion_Sunday['week date']=df_Rewards_Promotion_Sunday['week date'].astype(str)


df_Rewards_Promotion_Saturday=df_Rewards_Promotion[df_Rewards_Promotion['weekday']==5]
df_Rewards_Promotion_Saturday['Saturday_rewards_ind']=1
del df_Rewards_Promotion_Saturday['weekday']
df_Rewards_Promotion_Saturday['week date']=df_Rewards_Promotion_Saturday['week date'].astype(str)


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/

In [39]:
holiday_1_week_df=pd.DataFrame({"week date":[datetime.date(2016,12,18),datetime.date(2017,12,17),datetime.date(2018,12,23)],"Holiday_1_week_only_Ind":[1]*3},index=[0,1,2])
holiday_5_weeks_df=pd.DataFrame({"week date":[datetime.date(2016,12,18)-datetime.timedelta(days=x*7) for x in range(5)] +[datetime.date(2017,12,17)-datetime.timedelta(days=x*7) for x in range(5)]+[datetime.date(2018,12,23)-datetime.timedelta(days=x*7) for x in range(5)],
                                 "Holiday_5_weeks_only_Ind":[1]*15},index=[x for x in range(15)])

holiday_1_week_df['week date']=holiday_1_week_df['week date'].astype(str)
holiday_5_weeks_df['week date']=holiday_5_weeks_df['week date'].astype(str)

df_special_weeks_ind=pd.merge(df_Rewards_Promotion_Saturday,df_Rewards_Promotion_Sunday,on="week date",how="outer")
df_special_weeks_ind=pd.merge(df_special_weeks_ind,holiday_1_week_df,on="week date",how="outer")
df_special_weeks_ind=pd.merge(df_special_weeks_ind,holiday_5_weeks_df,on="week date",how="outer")


In [40]:
data_media_national_wide=pd.merge(data_media_national_wide,df_special_weeks_ind,on="week date",how="left")

data_submedia_national_wide=pd.merge(data_submedia_national_wide,df_special_weeks_ind,on="week date",how="left")

data_media_national_wide=data_media_national_wide.fillna(0)
data_submedia_national_wide=data_submedia_national_wide.fillna(0)

In [41]:
data_media_national_wide.to_csv("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2019Q2/output/BL_MMM_data_media_national_wide_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
data_submedia_national_wide.to_csv("/home/jian/Projects/Big_Lots/TMR/TMR_data/Up_to_2019Q2/output/BL_MMM_data_submedia_national_wide_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

# Note email cpm updated based on the Q2 client number
# Flipp hosted seperated in wide version
# Circ # is based on internal and different from Dom's 